In [1]:
import numpy as np

from keras.models import Sequential, load_model
from keras.layers import Embedding, LSTM, Dense, Dropout
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
# read in data
fulltext = list(np.load('data//numpy//fullstrings.npy'))
titles = list(np.load('data//numpy//titles.npy'))
c2i = np.load('data//numpy//c2i.npy').item()
i2c = np.load('data//numpy//i2c.npy').item()

In [3]:
# set parameters

DROP_RATE = 0.33 # dropout
EMBEDDING_SIZE = 256 # embedding size
HIDDEN_SIZE = 256 # lstm feature vector
HIDDEN_LAYERS = 2 # number of layers
START_EPOCH = 0
VOCAB_SIZE = len(c2i.keys()) # number of characters

WINDOW_SIZE = 20 # context length
NUM_EPOCHS = 1000

OUT_INCREMENT = 10 # printout after n batches - and save

In [11]:
def cardGenerator(cardtext, title, windowsize, c2i=c2i, debug=False):
        
    i = 0
    indices = list(np.random.permutation(len(cardtext)))
    idx = indices[i]

    # pregenerate warmup sequence
    warmup = ''
    while len(warmup) < windowsize:
        warmup += title[idx]
    
    warmup = list(warmup[-windowsize:])
    
    # sequence start
    sequence = list(cardtext[idx])
    
    # create matrix
    x = []
    y = []
    
    sequence = warmup + sequence
    
    if debug:
        sequence = sequence
    else:
        sequence = [c2i[c] for c in sequence]
    
    # main iterator
    while True:
        
        # generate batch (of cards_per_batch cards)
        while len(sequence) > windowsize:
            x.append(np.array(sequence[:windowsize]))
            y.append(sequence[windowsize])
            sequence.pop(0)
        
        # generate batch_size worth of window-shifted data
        # reshape for sparse_categorical_crossentropy
        sequence = []
        y = np.array(y)
        y = y[:, np.newaxis]
        # yield and reset
        yield(np.asarray(x), y)
        x, y = [], []
        
        # check for too long, reset
        if i+1 >= len(indices):
            indices = np.random.permutation(len(cardtext))
            i = 0
            idx = indices[i]
        else:
            i += 1
            idx = indices[i]
            
        # pregenerate new warmup sequence
        warmup = ''
        while len(warmup) < windowsize:
            warmup += title[idx]

        warmup = list(warmup[-windowsize:])

        # sequence start
        sequence = list(cardtext[idx].replace('\ufeff',''))

        # create matrix
        x = []
        y = []

        sequence = warmup + sequence

        if debug:
            sequence = sequence
        else:
            sequence = [c2i[c] for c in sequence]

In [12]:
getbatch = cardGenerator(fulltext, titles, WINDOW_SIZE)

In [6]:
# define model
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBEDDING_SIZE, 
                    batch_input_shape=(1, WINDOW_SIZE, )))
model.add(Dropout(DROP_RATE))
for _ in range(HIDDEN_LAYERS-1):
    model.add(LSTM(HIDDEN_SIZE, return_sequences=True, stateful=True))
model.add(LSTM(HIDDEN_SIZE, stateful=True))
model.add(Dense(VOCAB_SIZE, activation='softmax'))

In [7]:
# compile
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd', metrics=['accuracy'])

In [8]:
# predict 'Ⓔ'

def predict(startchars='none', temperature=1.0, maxlen=1000):
    
    seq_out = []
    
    if temperature=='random':
        tmp = np.random.random()
    else:
        tmp = temperature
    
    # starting sequence
    if startchars=='none':
        seq_in = [c2i['Ⓔ'] for i in range(WINDOW_SIZE)]
    
    elif startchars=='random':
        seq_in = []
        alpha = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k',
                 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
                 'w', 'x', 'y', 'z']
        alpha = [a for a in alpha if a in c2i.keys()]
        while len(seq_in) < WINDOW_SIZE-1:
            rnd = np.random.randint(0, len(alpha))
            seq_in += [c2i[alpha[rnd]]]
        seq_in += [c2i['Ⓔ']]
    
    else:
        s = list(startchars)
        s = s[:WINDOW_SIZE]
        seq_out =  [c2i[c] for c in s]
        while len(s) < WINDOW_SIZE:
            s.insert(0, 'Ⓔ')
        seq_in = [c2i[c] for c in s]
        
    # softmax temperature
    # scaling factor of logits = logits/temperature
    # high temp = more confident = more diverse, more mistakes
    # low temp: more conservative
    # https://stackoverflow.com/questions/37246030/how-to-change-the-temperature-of-a-softmax-output-in-keras/37254117#37254117
    def sample(a, temperature=tmp):
        a = np.array(a)**(1/temperature)
        p_sum = a.sum()
        sample_temp = a/p_sum 

        # stupid fix for > 1 error
        while sum(sample_temp) > 1:
            sample_temp[0] -= 0.0001

        return np.argmax(np.random.multinomial(1, sample_temp, 1))

    for i in range(maxlen):

        # predict next char
        pred_out = model.predict(np.array(seq_in).reshape((1, WINDOW_SIZE)))
        # get index of highest pred
        idx = sample(pred_out[0])
        # save index for decoding
        seq_out.append(idx)
        # add index to input sequence
        seq_in.append(int(idx))
        # remove earliest
        seq_in.pop(0)

    # decode final sequence
    card_char = ''.join([i2c[int(i)] for i in seq_out])
    
    print(card_char)
    
    return card_char

In [13]:
# load model
model.load_weights('model/eduthesis-modelweights-epoch1-cards90.h5')
START_EPOCH = 0

In [14]:
# just train, fix epochs later

# save card predictions
predictions = []

for epoch_idx in range(START_EPOCH, NUM_EPOCHS):
    
    # go through all cards each epoch
    for batch in tqdm(range(int(len(fulltext)))):
        
        # get batch
        x_batch, y_batch = next(getbatch)
        
        # fit to card batch
        r = model.fit(x_batch, y_batch, epochs=2, batch_size=1, shuffle=False, verbose=0)

        # reset state
        model.reset_states()
        
        # if batch % OUT_INCREMENT == 0 and batch > 0:

        if batch % OUT_INCREMENT == 0 and batch > 0:
            model.save_weights('model/eduthesis-modelweights-epoch{}-cards{}.h5'.format(epoch_idx+1, batch))
            print("EPOCH:", epoch_idx+1, "card #:", batch, "of", len(fulltext), r)
            predict(startchars='random', temperature='random')
            predict(startchars='random', temperature='random')
            predict(startchars='random', temperature='random')
            print('\n\n')


  0%|          | 0/118 [00:00<?, ?it/s]
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/derek/anaconda3/envs/kerasCRF/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/derek/anaconda3/envs/kerasCRF/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/derek/anaconda3/envs/kerasCRF/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

  8%|▊         | 10/118 [08:47<1:35:00, 52.78s/it]

EPOCH: 1 card #: 10 of 118 <keras.callbacks.History object at 0x7fce24525ac8>
 a pradings an resports bale intertite the conterved and the on preports the progols of the porticul a listruds of conts teachers edearations and in onterts in the pertecture they intervitune in the schools, in the pridesstions.ⒺⒺⒺ, the stude the spofsictions and their clodides and intervelts of pomticul contrications, were in thege leadershing a progersicaled scied then seling the supports. and intergsions, in there wer schools, and intliditations, scon teachers in there splecticuls, stuy portent they speciticulllides, the in there dearonces. the polticlations, ban and and internities in teachers in the scudices in the pomts teachers in the study of develors, and sciences teachers in the classormed the sepported and the stuly scoul teachers identity in thering in their teachers in the reseltions, and scoring a leaders in the scifically, developpenters, as the suctions, and interntitutions and they conded tea

  9%|▉         | 11/118 [10:08<1:38:36, 55.29s/it]

 sconded and teachers in they interith, and teachers in their newirg of teachers in the scon the desports teachers in the supported and in their schools. and scienticy in the in ther pladications, and teachers in their schools of teachers in teachers, in thery identitions and teachers in teachers in teachers in netities in ther scourning in there schools, and in there sucoling schools, in the ponticul in their educations in thee suctions, an prostrations, and in ther scon in their in the devesoctions, and the portications, in teachers in the student in their entervities and in their in their cladications in their un leartics and teachers scuvionthiclentsive of they resports of bers in there schools, and teachers in the sulers the scind in their education in their the perportical, and deares of they schools, and in ther schoolss in the peraditity and the scups in encations. in there sclools, in teachers in the classrooms, withen cuntications in then schools, and scientity in enterations

 17%|█▋        | 20/118 [17:35<1:26:11, 52.77s/it]

...

EPOCH: 1 card #: 110 of 118 <keras.callbacks.History object at 0x7fce5e170748>
 new teacher support teachers and these problems to teacher support to schools.ⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺⒺ

 94%|█████████▍| 111/118 [1:39:26<06:16, 53.75s/it]

 clable schools that partentics teacher problems and that functing sciemns, a well or study-of classroom unicewed a not can be way process by the exöan-and improvement to analysis support knowledge rescasting this study and parbing the study basey fement to analysis offers. hidd these problem, murbations on and to areas wamed this applied crastryod exhands the presents strateghing study enactional mothen, as and that levels. we scient nepboted a nevist and stundars, that aldie schools between and utboos, this article effnsice this article enearch to a complex schools while thee isconsibe taachies of improvement a one goalls to teacher schools. this and improvement that than into thste proces. this analyze concepten its a compsing stunded to areas of analyzes knowledge base enacting enacting mmesting teacher student study in intolled that lole improvement is shiffically socior vising offers of process nurvide to those noting schools, achieve the encalorators and problem. ïut improvement

  0%|          | 0/118 [00:00<?, ?it/s]


IndexError: list index out of range

In [ ]:
# # todo: just save to json one time
# model.save('model/100test_model.h5')
# print("saved model to disk\n")
# model.save_weights('model/100test_model_weights.h5')
# print("saved model weights to disk\n")

In [ ]:
# # load model
# model.load_weights('model/v2-modelweights-epoch4-cards1000.h5')
# START_EPOCH = 25

In [ ]:
predict(startchars='random', temperature='random', maxlen=2000)